In [1]:
import pickle
with open('transcripts.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

FileNotFoundError: [Errno 2] No such file or directory: 'transcripts.pkl'

In [2]:
with open('transcripts_timestamps.pkl', 'rb') as infile:
     transcript_timestamps = pickle.load(infile)


In [3]:
with open('transcripts_timestamps_links.pkl', 'rb') as infile:
     transcript_links = pickle.load(infile)


In [4]:
import json
from collections import defaultdict
def chunk_records_by_title(data, time_gap=30):
    # Group records by Title
    records_by_title = defaultdict(list)
    for record in data:
        records_by_title[record['Title']].append(record)
    
    merged_records = []
    
    # Process each group of records with the same Title
    for title, records in records_by_title.items():
        # Sort records by timestamp to ensure they are in order
        records = sorted(records, key=lambda x: x['TimeStamp'])
        
        # Initialize the first chunk
        current_chunk = {
            'Title': records[0]['Title'],
            'Text': records[0]['Text'],
            'Start_TimeStamp': records[0]['TimeStamp'],
            'End_TimeStamp': records[0]['TimeStamp'],
            'link': records[0]['link']
        }
        
        for i in range(1, len(records)):
            current_record = records[i]
            
            # Check if current record's timestamp is within the allowed time gap from the start timestamp
            if current_record['TimeStamp'] - current_chunk['Start_TimeStamp'] <= time_gap:
                # Merge with current chunk
                current_chunk['Text'] += ' ' + current_record['Text']
                current_chunk['End_TimeStamp'] = current_record['TimeStamp']
            else:
                # Save the current chunk and start a new one
                merged_records.append(current_chunk)
                current_chunk = {
                    'Title': current_record['Title'],
                    'Text': current_record['Text'],
                    'Start_TimeStamp': current_record['TimeStamp'],
                    'End_TimeStamp': current_record['TimeStamp'],
                    'link': current_record['link']
                }
        
        # Append the last chunk for the current title
        merged_records.append(current_chunk)
    
    return merged_records

# Define the time gap threshold (in seconds)
time_gap = 30  # Adjust this value as needed

# Merge the records
merged_records = chunk_records_by_title(transcript_links, time_gap)



In [5]:
merged_records

[{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",
  'Start_TimeStamp': 0.719,
  'End_TimeStamp': 30.039,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s'},
 {'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': 'use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm zoom camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos o

In [36]:
# import openai
# len(transcript_links)

13870

In [17]:
# import ast
# test = ast.literal_eval(test)

In [18]:
# def sanitize_text(text):
#     return text.encode('utf-8', 'ignore').decode('utf-8')

In [6]:
grouped_data = {}

# Iterate through each dictionary in the data list
for record in merged_records:
    title = record['Title']
    if title not in grouped_data:
        grouped_data[title] = []  # Create a new list for each unique title
    grouped_data[title].append(record)  # Add the record to the corresponding list

# Convert the dictionary to a list of lists
list_of_lists = list(grouped_data.values())

# Print the result
print(list_of_lists)

[[{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag', 'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to", 'Start_TimeStamp': 0.719, 'End_TimeStamp': 30.039, 'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s'}, {'Title': 'llm zoomcamp 1.1 - introduction to llm and rag', 'Text': 'use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm zoom camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers 

In [23]:
def enrich_content(list_of_lists):
    # Iterate through each list in list_of_lists
    for sublist in list_of_lists:
        # Iterate through each dictionary in the sublist by index
        for i in range(len(sublist)):
            # Get previous, current, and next Text values
            prev_text = sublist[i - 1]['Text'] if i > 0 else ""
            current_text = sublist[i]['Text']
            next_text = sublist[i + 1]['Text'] if i < len(sublist) - 1 else ""
            
            # Combine into enriched content
            enriched_content = f"{prev_text} {current_text} {next_text}".strip()
            
            # Add the new field to the current dictionary
            sublist[i]['enriched_content'] = enriched_content

# Apply the function to enrich the content in each dictionary
enrich_content(list_of_lists)

enriched_dicts = []
for sublist in list_of_lists:
    for item in sublist:
        enriched_dicts.append(item)

In [25]:

with open('enriched.json', 'w') as json_file:
    json.dump(enriched_dicts, json_file, indent=4)


In [8]:
def make_prompt():
    return """  
You are a data processing system enabled with AI that specializes in Retrieval Augmented Generation and LLM. The list of dictionaries provided by the user are each pieces of 
an entire video. These videos are part of a course that teaches how to create a system that uses RAGs retreived from Elasticsearch to feed additional information to an LLM in
order to imporve their accuracy. Looking at all of the dictionaries you will add another key called "keywords" which is a list of words which will enable better results in the
retreival of these documents from Elasticsearch. Add ONLY the keyword field to these dictionaries and ONLY return a list of the dictionaries without any additional text or 
comments.
"""

In [9]:
prompt = make_prompt()

In [10]:
prompt

'  \nYou are a data processing system enabled with AI that specializes in Retrieval Augmented Generation and LLM. The list of dictionaries provided by the user are each pieces of \nan entire video. These videos are part of a course that teaches how to create a system that uses RAGs retreived from Elasticsearch to feed additional information to an LLM in\norder to imporve their accuracy. Looking at all of the dictionaries you will add another key called "keywords" which is a list of words which will enable better results in the\nretreival of these documents from Elasticsearch. Add ONLY the keyword field to these dictionaries and ONLY return a list of the dictionaries without any additional text or \ncomments.\n'

In [11]:
# import os
# from dotenv import load_dotenv
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [15]:
# from openai import OpenAI
# client = OpenAI()

# def add_keywords(data, prompt):
#     completion = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": f"{prompt}"},
#             {
#                 "role": "user",
#                 "content": f"{data}"
#             }
#         ]
#     )

#     return completion.choices[0].message

In [16]:
# dict_with_keywords = add_keywords(list_of_lists[0], prompt)

In [20]:
import ast
with open('ChatGPT_KeywordAdd.txt','w') as outfile:
    outfile.writelines(str(ast.literal_eval(dict_with_keywords.content)))

In [1]:
import ollama
def ask_ollama(prompt):
    response = ollama.generate(model='llama3', prompt = f"{prompt}")
    return response

In [2]:
def make_prompt(question):
    return f"""
Given the following list of dictionaries you are a system which extrapolates key words from lists of dictionaries and returns json format dictionaries. Your knowledge is 
specialized to Large Language Models, RAGs and other technology around these topics.

Could you please add keywords in a new field that will help with easier searching of these documents. Please return all of the following dictionaries in the data that follows,
but with a new field added "keywords" that will help searching them for relevant entries in Elasticsearch.

Also please correct any text in the text field that does not seem to be correct in the context of an LLM course. If for instance if you see "rack retrieval" please update it to
"RAG retrieval", "elastic search" to "Elasticsearch" etc.

AGAIN I JUST WANT ALL OF THE DATA RETURNED in JSON FORMAT BUT WITH A KEYWORD FIELD ADDED TO EACH DICTIONARY. And please do not return any extra wording with the response. Just
the dictionary with the corrections and extra field. Also please look at all the dictionaries together to try to determine the best keywords to use so that the documents can be
found easily. You should only return a list of dictionaries with the extra field. No other commentary or explanation. 

Data:
{question}
"""

In [27]:
import ast
added_keywords = []
bad_resp = []
for t in list_of_lists:
    for rec in t:
        try:
            added_keywords.append(ast.literal_eval(ask_ollama(make_prompt(rec))['response'])[0])
        except Exception as e:
            bad_resp.append((ask_ollama(make_prompt(rec))['response']))

In [26]:
added_keywords

[{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': "hi everyone welcome to our course this is our first module for the first unit so in this course we will learn about practical applications of large language models and particularly focus on RAG retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to",
  'Start_TimeStamp': 0.719,
  'End_TimeStamp': 30.039,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s',
  'keywords': ['LLM', 'RAG retrieval', 'Large Language Models']},
 {'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': 'introduction to Large Language Models and RAG retrieval techniques. This will be our running problem throughout the course.',
  'Keywords': ['Large Language Model', 'RAG retrieval'],
  'Start_TimeStamp': 32.16,
  'End_TimeStamp': 61.399,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=32

In [ ]:
filename = 'responses.json'

# Write the list of dictionaries to a JSON file
with open(filename, 'w') as json_file:
    json.dump(added_keywords, json_file, indent=4)

print(f"Responses saved to {filename}")

In [3]:
with_keywords = []
for i in list_of_lists[0]:
    with_keywords.append(ask_ollama(make_prompt(i))['response'])


NameError: name 'list_of_lists' is not defined

In [75]:
prompt = make_prompt(test)

# Send prompt to the Ollama model
# Example response handling (pseudo-code)
response = ollama.send_prompt(prompt)

# If the response is not valid JSON or missing keywords, print a fallback message
try:
    parsed_response = json.loads(response)
    for record in parsed_response:
        if 'keywords' not in record:
            print("The 'keywords' field is missing in some records.")
except json.JSONDecodeError:
    print("Response was not valid JSON. Please check the prompt and model setup.")

AttributeError: module 'ollama' has no attribute 'send_prompt'

In [73]:
json.dumps(test, indent=2)

'[\n  {\n    "Title": "llm zoomcamp 1.1 - introduction to llm and rag",\n    "Text": "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i\'ll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",\n    "Start_TimeStamp": 0.719,\n    "End_TimeStamp": 30.039,\n    "link": "https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s"\n  },\n  {\n    "Title": "llm zoomcamp 1.1 - introduction to llm and rag",\n    "Text": "use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm zoom camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that 

In [69]:
test

[{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",
  'Start_TimeStamp': 0.719,
  'End_TimeStamp': 30.039,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s'},
 {'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': 'use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm zoom camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos o

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("YouTube_API_KEY")

from googleapiclient.discovery import build

def get_video_metadata(api_key, video_id):
    # Create a YouTube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Request to get video details
    request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_id
    )
    
    # Execute the request
    response = request.execute()
    
    # Check if the response contains items
    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]  # Return the metadata of the first item (video)
    else:
        return None  # No video found

# Replace 'YOUR_API_KEY' with your actual API key
api_key = API_KEY
video_id = 'Q75JgLEXMsM'  # Replace with your actual video ID

video_metadata = get_video_metadata(api_key, video_id)

if video_metadata:
    print("Video Metadata:")
    print(f"Title: {video_metadata['snippet']['title']}")
    print(f"Description: {video_metadata['snippet']['description']}")
    print(f"Channel Title: {video_metadata['snippet']['channelTitle']}")
    print(f"Published At: {video_metadata['snippet']['publishedAt']}")
    print(f"View Count: {video_metadata['statistics']['viewCount']}")
    print(f"Like Count: {video_metadata['statistics'].get('likeCount', 'N/A')}")
    print(f"Dislike Count: {video_metadata['statistics'].get('dislikeCount', 'N/A')}")
    print(f"Comment Count: {video_metadata['statistics'].get('commentCount', 'N/A')}")
else:
    print("No video found or API error occurred.")


Video Metadata:
Title: LLM Zoomcamp 1.1 - Introduction to LLM and RAG
Description: Welcome to the first module of our course, LLM Zoomcamp! We cover the applications of LLM, focusing on RAG: retrieval augmented generation. Throughout the course, we will build a Q&A system using the FAQ data from our courses. We don't cover the theory behind LLMs, but we will learn how to utilize them effectively.

Timecodes:

00:00 Introduction to LLM Zoomcamp
04:03 Understanding LLMs
09:15 Exploring RAG

Free LLM course: https://github.com/DataTalksClub/llm-zoomcamp

Join DataTalks.Club: https://datatalks.club/slack.html
Our events: https://datatalks.club/events.html
Channel Title: DataTalksClub ⬛
Published At: 2024-06-10T15:59:30Z
View Count: 11944
Like Count: 242
Dislike Count: N/A
Comment Count: 16


In [4]:
import re
from googleapiclient.discovery import build

def get_video_metadata(api_key, video_id):
    """
    Retrieves metadata for a given YouTube video ID.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Request to get video details
    request = youtube.videos().list(
        part='snippet',
        id=video_id
    )
    
    # Execute the request
    response = request.execute()
    
    # Check if the response contains items
    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]  # Return the metadata of the first item (video)
    else:
        return None  # No video found

def extract_timecodes_and_descriptions(description):
    """
    Extracts timecodes and their associated descriptions from the video description using regex.
    """
    # Regex pattern to match timecodes in the format MM:SS or HH:MM:SS
    timecode_pattern = r"(\d{1,2}:\d{2}(?::\d{2})?)\s+(.+)"
    
    # Find all matches
    matches = re.findall(timecode_pattern, description)
    
    # Extracted timecodes and descriptions
    timecodes = [(match[0], match[1].strip()) for match in matches]
    
    return timecodes

def create_timestamp_dicts(video_id, video_metadata, timecodes):
    """
    Creates a list of dictionaries for each timecode, including title, timecode, text, description, and link.
    """
    base_url = f"https://www.youtube.com/watch?v={video_id}"
    
    timestamp_dicts = []

    for time_str, text in timecodes:
        # Convert timecode to seconds
        parts = time_str.split(":")
        if len(parts) == 2:
            minutes, seconds = map(int, parts)
            time_in_seconds = minutes * 60 + seconds
        elif len(parts) == 3:
            hours, minutes, seconds = map(int, parts)
            time_in_seconds = hours * 3600 + minutes * 60 + seconds
        else:
            continue  # Skip if timecode format is not recognized
        
        # Create a clickable link
        link = f"{base_url}&t={time_in_seconds}s"
        
        # Create a dictionary for each timestamp
        timestamp_dict = {
            'title': video_metadata['title'],
            'timecode': time_str,
            'text': text,
            'description': video_metadata['description'],
            'link': link
        }
        
        timestamp_dicts.append(timestamp_dict)
    
    return timestamp_dicts

def get_video_info_and_timestamps(api_key, video_id):
    """
    Main function that retrieves video metadata and creates timestamp dictionaries.
    """
    # Fetch video metadata
    video_metadata_response = get_video_metadata(api_key, video_id)
    
    if not video_metadata_response:
        print("No video found or API error occurred.")
        return []
    
    # Extract relevant metadata
    video_metadata = {
        'title': video_metadata_response['snippet']['title'],
        'description': video_metadata_response['snippet']['description']
    }

    # Extract timecodes and descriptions dynamically from the video description
    timecodes = extract_timecodes_and_descriptions(video_metadata['description'])
    
    # Generate dictionaries for each timecode
    timestamp_dicts = create_timestamp_dicts(video_id, video_metadata, timecodes)
    
    return timestamp_dicts

# Example usage
api_key = API_KEY  # Replace with your actual API key
video_id = 'Q75JgLEXMsM'  # Replace with your actual video ID

# Get video information and timestamp dictionaries
timestamp_dicts = get_video_info_and_timestamps(api_key, video_id)

# Print the result
for timestamp_dict in timestamp_dicts:
    print(timestamp_dict)


{'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG', 'timecode': '00:00', 'text': 'Introduction to LLM Zoomcamp', 'description': "Welcome to the first module of our course, LLM Zoomcamp! We cover the applications of LLM, focusing on RAG: retrieval augmented generation. Throughout the course, we will build a Q&A system using the FAQ data from our courses. We don't cover the theory behind LLMs, but we will learn how to utilize them effectively.\n\nTimecodes:\n\n00:00 Introduction to LLM Zoomcamp\n04:03 Understanding LLMs\n09:15 Exploring RAG\n\nFree LLM course: https://github.com/DataTalksClub/llm-zoomcamp\n\nJoin DataTalks.Club: https://datatalks.club/slack.html\nOur events: https://datatalks.club/events.html", 'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s'}
{'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG', 'timecode': '04:03', 'text': 'Understanding LLMs', 'description': "Welcome to the first module of our course, LLM Zoomcamp! We cover the applications of 

In [5]:
timestamp_dicts

[{'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
  'timecode': '00:00',
  'text': 'Introduction to LLM Zoomcamp',
  'description': "Welcome to the first module of our course, LLM Zoomcamp! We cover the applications of LLM, focusing on RAG: retrieval augmented generation. Throughout the course, we will build a Q&A system using the FAQ data from our courses. We don't cover the theory behind LLMs, but we will learn how to utilize them effectively.\n\nTimecodes:\n\n00:00 Introduction to LLM Zoomcamp\n04:03 Understanding LLMs\n09:15 Exploring RAG\n\nFree LLM course: https://github.com/DataTalksClub/llm-zoomcamp\n\nJoin DataTalks.Club: https://datatalks.club/slack.html\nOur events: https://datatalks.club/events.html",
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s'},
 {'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
  'timecode': '04:03',
  'text': 'Understanding LLMs',
  'description': "Welcome to the first module of our course, LLM Zoomcamp! We cover the

In [4]:
import pickle
with open('transcripts_metadata_records.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

FileNotFoundError: [Errno 2] No such file or directory: 'transcripts_metadata_records.pkl'

In [11]:
for transcript in transcripts:
    transcript['text_vector'] = transcript['title']+' '+transcript['text']+' '+transcript['description']

In [14]:
len(transcripts[0]['text_vector'])

392

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-30 10:51:32.557295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 10:51:32.576583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 10:51:32.576616: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 10:51:32.589789: I tensorflow/core/platform/cpu_feature_gua

In [16]:
for transcript in transcripts:
    transcript['search_vector'] = model.encode(transcript['text_vector'])

In [19]:
transcripts[0].keys()

dict_keys(['title', 'timecode', 'text', 'description', 'link', 'text_vector', 'search_vector'])

In [20]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "timecode": {"type": "text"},
            "text": {"type": "text"},
            "description": {"type": "keyword"} ,
            "link": {"type": "keyword"} ,
            "text_vector": {"type": "keyword"} ,
            "search_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "video-content"

# es.indices.delete(index=index_name, ignore_unavailable=True)
# es.indices.create(index=index_name, body=index_settings)

In [43]:
for doc in transcripts:
    try:
        es.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [5]:

search_term = "When did we learn to evaluate RAGs?"
vector_search_term = model.encode(search_term)

In [7]:
query = {
    "field": "search_vector",
    "query_vector": vector_search_term,
    "k": 10,
    "num_candidates": 10000, 
}
res = es.search(index=index_name, knn=query, source=["title","link"])
res["hits"]["hits"]

[{'_index': 'video-content',
  '_id': '0X_9o5EBQoThIisVeRm_',
  '_score': 0.7103136,
  '_source': {'link': 'https://www.youtube.com/watch?v=IB6jePK1s58&t=850s',
   'title': 'LLM Zoomcamp 4.5 - Offline RAG Evaluation: LLM as a Judge'}},
 {'_index': 'video-content',
  '_id': 'y3_9o5EBQoThIisVeRmf',
  '_score': 0.70137733,
  '_source': {'link': 'https://www.youtube.com/watch?v=IB6jePK1s58&t=70s',
   'title': 'LLM Zoomcamp 4.5 - Offline RAG Evaluation: LLM as a Judge'}},
 {'_index': 'video-content',
  '_id': '0n_9o5EBQoThIisVeRnG',
  '_score': 0.6982006,
  '_source': {'link': 'https://www.youtube.com/watch?v=IB6jePK1s58&t=1000s',
   'title': 'LLM Zoomcamp 4.5 - Offline RAG Evaluation: LLM as a Judge'}},
 {'_index': 'video-content',
  '_id': 'yn_9o5EBQoThIisVeRmb',
  '_score': 0.6961205,
  '_source': {'link': 'https://www.youtube.com/watch?v=IB6jePK1s58&t=0s',
   'title': 'LLM Zoomcamp 4.5 - Offline RAG Evaluation: LLM as a Judge'}},
 {'_index': 'video-content',
  '_id': '1H_9o5EBQoThIisVeR

In [1]:
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-01 18:17:23.431806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 18:17:23.451876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 18:17:23.451908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 18:17:23.465016: I tensorflow/core/platform/cpu_feature_gua